# 处理模型方法数据，将其入库

数据：“./method-data/方法模型数据.xlsx”

## 读取excel


In [12]:
import pandas as pd 

df_method=pd.read_excel('./method-data/方法模型数据.xlsx',sheet_name='processed')
df_method.head(3)

,methodIdentifier,methodName/label,methodDescription,inputDataDescription,outputDataDescription,inputData,outputData,inputFormat,outputFormat,inputDataFormat1,inputDataFormat2,outputDataForamt1,outputDataForamt2,validationArea,methodType,belongToStage,addition
0,Method_TB1174,叠加分析,分析在空间位置上有一定关联的空间对象的空间特征和专属属性之间的相互关系。,多个矢量地理要素类,矢量地理要素类,GeologicalData,GeologicalData,Vector Data Format,Vector Data Format,Vector Data Format,NaN,Vector Data Format,NaN,NaN,GISMethod,response;recovery,NaN
1,Method_PM1601,空间连接,根据空间关系将一个要素类的属性连接到另一个要素类的属性。目标要素和来自连接要素的被连接属性写...,目标要素、连接要素、连接类型、连接操作、匹配选项等,包含目标要素和连接要素的属性的新要素类。默认情况下，目标要素的所有属性和被连接要素的属性都被...,GeologicalData,GeologicalData,Vector Data Format,Vector Data Format,Vector Data Format,NaN,Vector Data Format,NaN,NaN,GISMethod,NaN,NaN
2,Method_YJ6279,裁剪,提取与裁剪要素相重叠的输入要素。,裁剪要素可以是点、线和面，具体取决于输入要素的类型。,输出要素类将包含输入要素的所有属性。,GeologicalData,GeologicalData,Vector Data Format,Vector Data Format,Vector Data Format,NaN,Vector Data Format,NaN,NaN,GISMethod,NaN,NaN


## 新建一个ttl文件并写入前缀

In [3]:
# 创建一个ttl文件，文件名为filename,并向文件写入prefix
def ttl_create(filename, prefix):
    relative_path = "./rdf/"  # 新创建的txt文件的存放路径
    full_path = relative_path + filename + '.ttl'  
    file = open(full_path, 'w')
    file.write(prefix)   #msg也就是下面的Hello world!
    file.close()

name="模型方法实例"
prefix='''@prefix : <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schema.org/> .
@base <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology> .

'''

ttl_create(name, prefix)


由于模型方法实例中暂时没有需要提前创建的其他类型实例（如时间、空间），因此可以直接创建模型方法实例。

（验证区域validationArea的空间范围到最后再创建）


## 创建Method实例


In [13]:
def create_methodIndividual(df):
    text="";
    for index, row in df.iterrows():
        # print(index,row['中文名'].strip(),row['DisasterIdentifier']) 
        identifier=row['methodIdentifier']
        name=row['methodName/label']
        description=row['methodDescription']
        if not pd.isnull(description):
            description=description.replace('\n', '')
        mtype=row["methodType"]


        method_text=f'''### http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}
:{identifier} rdf:type owl:NamedIndividual ,:{mtype} ;
               :methodIdentifier "{identifier}" ;
               :modelDescription "{description}" ;
               :modelName "{name}" ;
               rdfs:label "{name}" .
'''
        inputdes=row["inputDataDescription"]
        outputdes=row["outputDataDescription"]
        inputData=row["inputData"]
        outputData=row["outputData"]
        inputFormat=row["inputFormat"]
        outputFormat=row["outputFormat"]
        validationArea=row["validationArea"]
        addition=row["addition"]
        stage=row["belongToStage"]

        if not pd.isnull(inputdes):
            method_text=method_text+f''':{identifier} :inputDataDescription "{inputdes}" .
'''
        if not pd.isnull(outputdes):
            method_text=method_text+f''':{identifier} :outputDataDescription "{outputdes}" .
'''        
        if not pd.isnull(inputData):
            inputData=inputData.replace(";",", :")
            method_text=method_text+f''':{identifier} :methodInput :{inputData} .
'''
        if not pd.isnull(outputData):
            outputData=outputData.replace(";",", :")
            method_text=method_text+f''':{identifier} :methodOutput :{outputData} .
'''      
        if not pd.isnull(inputFormat):
            inputFormat=inputFormat.replace(" ","").replace(";",", :")
            method_text=method_text+f''':{identifier} :methodInputFormat :{inputFormat} .
'''
        if not pd.isnull(outputFormat):
            outputFormat=outputFormat.replace(" ","").replace(";",", :")
            method_text=method_text+f''':{identifier} :methodOutputFormat :{outputFormat} .
'''  

        if not pd.isnull(validationArea):
            method_text=method_text+f''':{identifier} :validationArea "{validationArea}" .
'''      
        if not pd.isnull(addition):
            method_text=method_text+f''':{identifier} :additionalInfo "{addition}" .
'''
        if not pd.isnull(stage):
            stage=stage.replace(";",", :")
            method_text=method_text+f''':{identifier} :hasDevelopmentStage :{stage} .
'''  
        text=text+method_text
    return text

text= create_methodIndividual(df_method)
file = open("./rdf/模型方法实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()